In [1]:
import pandas as pd

In [5]:
movie_df = pd.read_csv('complete_df.csv')
movie_df = movie_df.drop(['role', 'mbti', 'danceability_mean', 'energy_mean', 'loudness_mean', 'mode_mean', 'speechiness_mean', 'acousticness_mean', 'liveness_mean', 'valence_mean', 'tempo_mean' , 'instrumentalness_mean'], axis=1)
print(movie_df)

                                       movie              processed_movie  \
0                  Marvel Cinematic Universe    marvel cinematic universe   
1                                  Star Wars                    star wars   
2                   Harry Potter (franchise)                 harry potter   
3                   Harry Potter (franchise)                 harry potter   
4                       The Lion King (1994)                the lion king   
...                                      ...                          ...   
17274          The Amazing Spider-Man (2012)       the amazing spider-man   
17275     The Girl with All the Gifts (2016)  the girl with all the gifts   
17276                          Eat Pray Love                eat pray love   
17277                           The Informer                 the informer   
17278                        Revolver (2005)                     revolver   

                   expanded-genres  \
0                              NaN   

In [18]:
import pandas as pd

# Define the dictionary
music_genres = {
    "Marvel Cinematic Universe": ['alternative', 'pop', 'punk', 'jazz'],
    "Harry Potter (franchise)": ['indie', 'rock', 'pop'],
    "DC Extended Universe": [ 'punk', 'ambient', 'dubstep', 'hard-rock', 'rock'],
    "Jurassic Park / Jurassic World": ['ambient', 'new-age', 'alternative', 'rock'],
    "X-Men Cinematic Universe (2011-2020)": ['acoustic', 'folk', 'punk', 'ambient', 'dubstep', 'hard-rock', 'ambient'],
    "The Dark Knight Trilogy": ['hip-hop', 'grime', 'trap', 'drum-and-bass', 'dubstep', 'electronic', 'reggae'],
    "DC Animated Movie Universe": ['acoustic', 'folk', 'punk', 'ambient', 'dubstep', 'hard-rock', 'ambient'],
    "Pirates of the Caribbean": ['epic-fantasy', 'folk', 'ambient', 'dubstep', 'indie', 'rock', 'pop'],
    "Terminator": ['synthwave', 'synth-pop', 'electronic', 'ambient', 'chill'],
    "The Hobbit": ['epic-fantasy', 'folk', 'ambient', 'dubstep', 'indie', 'rock', 'pop'],
    "Indiana Jones": ['world-music', 'soundtrack', 'patriotic', 'rock', 'metal', 'punk', 'alternative'],
    "The Spider-Man Trilogy": ['world-music', 'soundtrack', 'patriotic', 'rock', 'metal', 'punk', 'alternative'],
    "Bill & Ted": ['acoustic', 'folk', 'punk', 'ambient', 'dubstep', 'hard-rock', 'ambient'],
    "Police Academy": ['hip-hop', 'grime', 'trap', 'drum-and-bass', 'dubstep', 'electronic', 'reggae'],
    "Wreck-it Ralph (Franchise)": ['epic-fantasy', 'folk', 'ambient', 'dubstep', 'indie', 'rock', 'pop'],
    "Maze Runner": ['synthwave', 'synth-pop', 'electronic', 'ambient', 'chill', 'space', 'retro'],
    "Dead Poets Society (1989)": ['acoustic', 'piano', 'singer-songwriter', 'indie', 'indie-pop', 'alternative', 'rock'],
    "Mean Girls (2004)": ['pop', 'r-n-b', 'soul', 'singer-songwriter', 'indie'],
    "Percy Jackson (film series 2010-2013)": ['folk', 'ambient', 'dubstep', 'indie', 'rock', 'pop'],
    "Ocean's 8 (2018)": ['jazz', 'r-n-b', 'soul', 'hip-hop'],
    "Lady Bird (2017)": ['alternative', 'singer-songwriter', 'indie', 'punk', 'romantic'],
    "Boss Baby (2017)": ['children', 'pop', 'happy'],
    "The Perks of Being a Wallflower (2012)": ['sad', 'acoustic', 'singer-songwriter', 'indie', 'indie-pop', 'alternative', 'rock'],
    "Booksmart (2019)": ['pop', 'punk', 'disco', 'singer-songwriter', 'indie'],
    "Once Upon a Time in Hollywood (2019)": ['jazz', 'r-n-b', 'soul', 'singer-songwriter', 'indie', 'folk', 'romantic'],
    "The King of Staten Island (2020)": ['rock', 'alternative', 'soul', 'singer-songwriter', 'indie'],
    "Eighth Grade (2018)": ['singer-songwriter', 'indie', 'folk'],
    "Mixed Nuts": ['pop', 'r-n-b', 'soul', 'singer-songwriter', 'indie', 'folk'],
    "Step Brothers (2008)": ['pop', 'r-n-b', 'soul', 'singer-songwriter', 'indie', 'folk'],
    "The Hangover Trilogy": ['pop', 'hip-hop', 'dubstep', 'singer-songwriter', 'indie']
}

# Convert dictionary to DataFrame
df = pd.DataFrame(list(music_genres.items()), columns=['Movies', 'Genres'])

# Remove square brackets around genres
df['Genres'] = df['Genres'].apply(lambda x: ', '.join(x))

# Display the DataFrame
print(df)


                                    Movies  \
0                Marvel Cinematic Universe   
1                 Harry Potter (franchise)   
2                     DC Extended Universe   
3           Jurassic Park / Jurassic World   
4     X-Men Cinematic Universe (2011-2020)   
5                  The Dark Knight Trilogy   
6               DC Animated Movie Universe   
7                 Pirates of the Caribbean   
8                               Terminator   
9                               The Hobbit   
10                           Indiana Jones   
11                  The Spider-Man Trilogy   
12                              Bill & Ted   
13                          Police Academy   
14              Wreck-it Ralph (Franchise)   
15                             Maze Runner   
16               Dead Poets Society (1989)   
17                       Mean Girls (2004)   
18   Percy Jackson (film series 2010-2013)   
19                        Ocean's 8 (2018)   
20                        Lady Bir

In [24]:
# Merge the two DataFrames on the processed movie titles
merged_df = movie_df.merge(df, left_on='processed_movie', right_on='Movies', how='left')

# Fill in the 'Associated Music Genres' column with values from the merged 'Genres' column
merged_df['Associated Music Genres'] = merged_df.apply(lambda x: x['Associated Music Genres'] if pd.notnull(x['Associated Music Genres']) else x['Genres'], axis=1)

# Drop unnecessary columns from the merged DataFrame
merged_df.drop(columns=['Movies', 'Genres', 'processed_movie'], inplace=True)

# Display the merged DataFrame
print(merged_df)


                                       movie              expanded-genres  \
0                  Marvel Cinematic Universe                          NaN   
1                                  Star Wars   Action, Adventure, Fantasy   
2                   Harry Potter (franchise)                          NaN   
3                   Harry Potter (franchise)                          NaN   
4                       The Lion King (1994)  Animation, Adventure, Drama   
...                                      ...                          ...   
17274          The Amazing Spider-Man (2012)                          NaN   
17275     The Girl with All the Gifts (2016)     Action, Adventure, Drama   
17276                          Eat Pray Love    Biography, Drama, Romance   
17277                           The Informer         Action, Crime, Drama   
17278                        Revolver (2005)         Action, Crime, Drama   

                                 Associated Music Genres  
0               

In [12]:
nan_count = movie_df['Associated Music Genres'].isna().sum()

print("Number of NaN values in the 'Associated Music Genres' column:", nan_count)
nan_rows = movie_df[movie_df['Associated Music Genres'].isna()]

# Count occurrences of each movie
movie_counts = nan_rows['movie'].value_counts()

print("Movies with the most occurrences in the DataFrame where 'Associated Music Genres' column contains NaN values:")
for movie, count in movie_counts.items():
    print(movie, ":", count)

Number of NaN values in the 'Associated Music Genres' column: 4305
Movies with the most occurrences in the DataFrame where 'Associated Music Genres' column contains NaN values:
   Marvel Cinematic Universe : 175
   Harry Potter (franchise) : 83
   DC Extended Universe : 60
   Jurassic Park / Jurassic World : 55
   X-Men Cinematic Universe (2011-2020) : 41
   The Dark Knight Trilogy : 40
   The Shawshank Redemption (1994) : 40
   DC Animated Movie Universe : 40
   Pirates of the Caribbean : 39
   Terminator : 34
   The Hobbit : 32
   The Matrix Trilogy (The Matrix) : 27
   The Lego Movies : 26
   The Red Balloon (1956) : 25
   The Chronicles of Narnia (Film Trilogy) : 23
   Tinker Bell (2008) : 23
   Fantastic Beasts : 22
   G.I. Joe (film series) : 22
   American Pie : 21
   Descendants : 20
   Indiana Jones : 20
   The Spider-Man Trilogy : 19
   Bill & Ted : 19
   Hababam Sınıfı (1975) : 19
   Police Academy : 19
   X-Men Cinematic Universe (2000-2009) : 19
   Short Cuts (1993) : 18
 

In [26]:
movie_df = movie_df.drop_duplicates(subset='movie')
print(movie_df)

                                     movie            processed_movie  \
0                Marvel Cinematic Universe  marvel cinematic universe   
1                                Star Wars                  star wars   
2                 Harry Potter (franchise)               harry potter   
4                     The Lion King (1994)              the lion king   
5                        Shrek (Franchise)                      shrek   
...                                    ...                        ...   
17215                          The Killers                the killers   
17229                   Logan's Run (1976)                logan's run   
17251     Millennium series (Swedish 2009)          millennium series   
17254                   The 6th Day (2000)                the 6th day   
17270                    The Switch (2010)                 the switch   

                    expanded-genres  \
0                               NaN   
1        Action, Adventure, Fantasy   
2     

In [29]:
movie_df.to_csv('movie_df.csv', index=False)

In [16]:
genres = pd.read_csv('genre_avg_metrics.csv')
for value in genres['track_genre']:
    print(value)

acoustic
afrobeat
alt-rock
alternative
ambient
anime
black-metal
bluegrass
blues
brazil
breakbeat
british
cantopop
chicago-house
children
chill
classical
club
comedy
country
dance
dancehall
death-metal
deep-house
detroit-techno
disco
disney
drum-and-bass
dub
dubstep
edm
electro
electronic
emo
folk
forro
french
funk
garage
german
gospel
goth
grindcore
groove
grunge
guitar
happy
hard-rock
hardcore
hardstyle
heavy-metal
hip-hop
honky-tonk
house
idm
indian
indie
indie-pop
industrial
iranian
j-dance
j-idol
j-pop
j-rock
jazz
k-pop
kids
latin
latino
malay
mandopop
metal
metalcore
minimal-techno
mpb
new-age
opera
pagode
party
piano
pop
pop-film
power-pop
progressive-house
psych-rock
punk
punk-rock
r-n-b
reggae
reggaeton
rock
rock-n-roll
rockabilly
romance
sad
salsa
samba
sertanejo
show-tunes
singer-songwriter
ska
sleep
songwriter
soul
spanish
study
swedish
synth-pop
tango
techno
trance
trip-hop
turkish
world-music
